In [8]:
from PIL import Image
import os
import cv2
import numpy as np

def crop_image(image_path, left, top, right, bottom):
    image = Image.open(image_path)
    cropped_image = image.crop((left, top, right, bottom))

    return cropped_image

def generate_images(image_path, rotation_angles, output_path):
    # Load the original image
    original_image = Image.open(image_path)
    image_name = os.path.splitext(os.path.basename(image_path))[0]

    # Iterate through the rotation angles
    for angle in rotation_angles:
        rotated_image = original_image.rotate(angle, expand=True, fillcolor='white')
        saved_path = f"{output_path}{image_name}__rot{angle}deg.png"
        rotated_image.save(saved_path)

    # Save images for Gaussian Noise
    image = cv2.imread(image_path)

    # Generate random Gaussian noise
    mean = 0
    stddev = 750
    noise = np.zeros_like(image, dtype=np.int16)
    cv2.randn(noise, mean, stddev)

    # Add the noise to the image
    noisy_image = np.clip(image + noise, 0, 255).astype(np.uint8)

    # Save the noisy image
    cv2.imwrite(f'{output_path}{image_name}__gnoise.png', noisy_image)

In [87]:
angles = [-5, -7.5, -10, -12.5, -15, -20, -22.5 -25, -27.5, -30, -32.5, -35, -37.5, -40, -42.5, -45, 45, 42.5, 40, 37.5, 35, 32.5, 30, 27.5, 25, 22.5, 20, 15, 12.5, 10, 7.5, 5]

path = "./dataset/Zyad/"

for img_name in os.listdir(path):
    generate_images(image_path=path+img_name, rotation_angles=angles, output_path=path)

In [88]:
# Angles * Image numbers (50) + Noise (1 noise per image = 50 images) + Number of ori images (50)

len(angles) * 50 + 50 + 50

1650

In [89]:

# Get the list of files and directories in the specified directory
file_list = os.listdir(path)

# Count the number of files
file_count = len([file for file in file_list if os.path.isfile(os.path.join(path, file))])

print("Number of files:", file_count)

Number of files: 1650


# Making Image Decoding Faster

In [12]:
import torch
from torchvision.datasets import DatasetFolder
import torchvision.transforms as transforms
from PIL import Image

import os

data_dir = './dataset'
output_dir = './processed_dataset_flattened'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

transform = transforms.Compose([
    # transforms.Lambda(lambda img: img.convert('RGB')),  # Convert image to RGB format
    transforms.Grayscale(),
    transforms.Resize((224, 224)),  # Resize images to (224, 224)
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize((0.5,), (0.5,)),  # Normalize the image tensors
    transforms.Lambda(lambda x: x.view(-1))  # Flatten the tensor
])

# Iterate over the class folders and save processed tensors
for class_folder in os.listdir(data_dir):
    class_path = os.path.join(data_dir, class_folder)
    if os.path.isdir(class_path):
        class_output_dir = os.path.join(output_dir, class_folder)
        os.makedirs(class_output_dir, exist_ok=True)

        for file in os.listdir(class_path):
            if file.endswith('.png'):
                file_path = os.path.join(class_path, file)
                output_path = os.path.join(class_output_dir, os.path.splitext(file)[0] + '.pt')

                # Load and preprocess the image
                image = transform(Image.open(file_path))

                # Save the processed tensor as .pt file
                torch.save(image, output_path)
